In [1]:
from __future__ import absolute_import, division

import Model
import numpy as np 
#import preTool
from keras.preprocessing.sequence import pad_sequences
import senteval
import pickle
import copy
import operator
import nltk
from keras.layers import Dense, Bidirectional,Input, Dropout, Flatten, concatenate, dot, GaussianDropout, Activation, GRU, Embedding
from keras.models import Model
from keras.regularizers import l1, l2



PATH_SENTEVAL = '../'
PATH_TO_DATA = 'data'

maxLength = 30    
wordDim = 300    # 300 
num_neurons = [600,600]   # [600,600f]
extractLayer = ['X_input_second','X_input_mask_2']
kernel_reg = 0.001
batch_size = 200
merge_mode = 'concat'
checkpoint =  "../randomInitialization.hdf5"  # Change 
embeddingPath = "../Embedding/word2vec.810B.300d.txt"  # Change 
processedPath = "../processed/"

textPath = "text2/"
wordDicFile = 'wordDic_case.txt'
tokenizerFile = 'tokenizer_case.txt'
embeddingFile =  'embedding_matrix_case.txt'
embeddingFastFile = 'embedding_matrix_fast'
valiFile = "val_set.txt"

model = 0
count = 0
NonExiste = {}

    
with open(processedPath + tokenizerFile , "rb") as input_file:
    t = pickle.load(input_file)
    
with open(processedPath + embeddingFile , "rb") as input_file:
    embedding_matrix = pickle.load(input_file)

with open(processedPath + embeddingFastFile    , "rb") as input_file:
    embedding_matrix_fast = pickle.load(input_file)
    
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 

# SentEval prepare and batcher
def prepare(params, samples):
    return
import copy

def batcher(params, batch):
    global model, count 
    sentenceList = []
    sentences = [' '.join(s) for s in batch]
    sentences_2 = copy.deepcopy(sentences)

    #embedding_matrix = np.zeros([49985,wordDim])
        
    sentences = preprocessing(sentences)
    sentences = t.texts_to_sequences(sentences)
    sentences = pad_sequences(sentences, maxlen=maxLength, padding='post')

    
    if model == 0 :
            print("Start loading model")
            model = buildModel(sentences,kernel_reg,num_neurons,merge_mode, len(embedding_matrix),maxLength,wordDim,embedding_matrix,embedding_matrix_fast ,False)
            print("Loading model")
            model.load_weights(checkpoint)
            print("Finish build model")
#                     buildModel(input_X_vali,kernel_reg,num_neurons,merge_mode,vocab_size,maxLength,wordDim,embedding_matrix,embedding_matrix_fast,weightTrainable)
    #print("Extract hidden ")
    output_model = extractHiddenState(extractLayer,model,sentences)
    output_model = np.mean(np.array(output_model),axis= 0, keepdims = True)
    
    
    for i,sentence in enumerate(sentences_2):
        
#         sentence = sentence.replace("n 't", "n't")
        #sentence = sentence.replace("-"," ")
        sentence =  nltk.word_tokenize(sentence)
        stopWord = []
        nonStopWord = []

        for j,vocab in enumerate(sentence):

            lower = vocab.lower() 
            if lower in stop_words:  # has to be lower case 
                try:
                    stopWord.append( list(output_model[0][i][j]))
#                     stopWord.append( list(output_model[0][i][j]) + list(output_model[1][i][j]) )
                except:
                    if vocab not in NonExiste.keys():
                        NonExiste[vocab] = 1
                    else: 
                        NonExiste[vocab] += 1 
            else:
                try:
                    nonStopWord.append(list(output_model[0][i][j]))
#                     nonStopWord.append(list(output_model[0][i][j]) + list(output_model[1][i][j]))
                except:
                    if vocab not in NonExiste.keys():
                        NonExiste[vocab] = 1
                    else: 
                        NonExiste[vocab] += 1  
        #print("Stopword:{}, nonStopword:{}, total: {}".format(len(stopWord),len(nonStopWord),len(sentence)))
        if len(stopWord) == 0:
            stopWord = np.zeros([1,wordDim*4])
        if len(nonStopWord) == 0:
            nonStopWord = np.zeros([1,wordDim*4])


        average = np.mean(np.array(list(stopWord) + list(nonStopWord)),axis = 0 , keepdims = True)
        stopWord = np.mean(np.array(stopWord),axis= 0, keepdims = True)   
        nonStopWord = np.mean(np.array(nonStopWord),axis= 0, keepdims = True)
        
#         min_ = np.amin(np.array(list(stopWord) + list(nonStopWord)), axis=0,keepdims = True)
#         max_ = np.amax(np.array(list(stopWord) + list(nonStopWord)), axis=0,keepdims = True)
#         output = (list(average[0]) + list(stopWord[0]) + list(nonStopWord[0])+list(min_[0]) +list(max_[0]))
        output = list(average[0]) + list(stopWord[0]) + list(nonStopWord[0])
        sentenceList.append(output)
        #print(len(output))
    return sentenceList


Using TensorFlow backend.


In [ ]:
tasks =  ['MRPC','CR','MR','SUBJ','MR','TREC'] #', 'CR','MPQA',
result = []
for task in tasks:
    print(task)
    
    params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10}
    params_senteval['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size':256,
                                     'tenacity': 5, 'epoch_size': 4}
    # Set up logger
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    transfer_tasks =    [task]  # ['CR','MPQA','TREC','MRPC']
    # 'MR', 'CR' ,'MPQA', 'SUBJ','TREC', 'MRPC','SICKEntailment', 'SICKRelatedness'  ,'TREC','MRPC'
    results = se.eval(transfer_tasks)
    print(results)
    result.append(results)

MRPC


In [ ]:
results

In [ ]:
output_model = extractHiddenState(extractLayer,model,sentence)

In [ ]:
np.array(output_model).shape
x = np.mean(np.array(output_model),axis= 0, keepdims = True)

In [ ]:
x[0].shape

In [2]:
def buildModel(input_X,kernel_reg,num_neurons,merge_mode, vocab_size,max_length, wordDim,embedding_matrix,embedding_matrix_2,trainable):
    # Create shared layer for masked sentence 
    embedding_1 = Embedding(vocab_size, wordDim, weights=[embedding_matrix], input_length=max_length, trainable=trainable ,name = "embedding")
    embedding_2 = Embedding(vocab_size, wordDim, weights=[embedding_matrix_2], input_length=max_length, trainable=trainable ,name = "embedding_fast")
    shared_second = Bidirectional(GRU(num_neurons[1],kernel_initializer = 'RandomUniform',return_sequences = True,kernel_regularizer=l2(kernel_reg)), merge_mode= merge_mode,name='X_input_mask_2')

    org_input = Input(shape=(input_X.shape[1],), name = 'org_input')
    emb_org = embedding_1(org_input)
    org_input_2 = Bidirectional(GRU(num_neurons[1],kernel_initializer = 'Orthogonal',return_sequences = True, kernel_regularizer=l2(kernel_reg)), merge_mode= merge_mode,name='X_input_second')(emb_org)

    mask_input_first = Input(shape=(input_X.shape[1],), name = 'mask_input_first')
    emb_first = embedding_2(mask_input_first)
    mask_input_first_2 = shared_second(emb_first)

    mask_input_second = Input(shape=(input_X.shape[1],), name = 'mask_input_second')
    emb_second = embedding_2(mask_input_second)
    mask_input_second_2 = shared_second(emb_second)

    mask_input_third = Input(shape=(input_X.shape[1],), name = 'mask_input_third')
    emd_third = embedding_2(mask_input_third)
    mask_input_third_2 = shared_second(emd_third)

    out1 = dot([org_input_2, mask_input_first_2], axes=1, normalize = True,name='output_1')
    out2 = dot([org_input_2, mask_input_second_2], axes=1,normalize = True,name='output_2')
    out3 = dot([org_input_2, mask_input_third_2], axes=1, normalize = True,name='output_3')

    concat = concatenate([out1, out2, out3], name='concat')

    output = Activation('softmax')(concat)

    model = Model(inputs=[org_input,mask_input_first,mask_input_second,mask_input_third], outputs=output)

    return model

def extractHiddenState(layerName,model,predict_input):
    
    outputs = []

    for i in range(len(layerName)):
        outputs.append(model.get_layer(layerName[i]).get_output_at(0))

    input_layer = Model(inputs=model.input,
                        outputs=outputs)

    input_layer_embed = input_layer.predict([predict_input,predict_input,predict_input,predict_input])
    
    return input_layer_embed

def preprocessing(lines): 
    #print("Pre processing Data")
  
    for line in lines:
        # Decapitalize: Conver to lower case first. 
        line = line.lower()
        # Delete url 
#         line = re.sub(r"http\S+", "link", line)
#         line = re.sub(r"\S+html", "link", line)
#         line = re.sub(r"\S+.com$", "link", line)
#         line = re.sub(r"\S+.jpg$", "photo", line)
#         #line = decontracted(line)
        '''
        ignore:  * { } \  < > 
        Splite based on : ! . ,  &  # ' $ 
        line = re.findall(r"[\w']+|[().,:!?;'$&]", line)
        '''
        #line = re.findall(r"[\w']+|[().,:!?;$&]", line)
        line = line.split()
 
    return lines



In [ ]:
results